<a href="https://colab.research.google.com/github/JohnSili/NovosibCFP/blob/main/%D1%86%D0%B8%D1%84%D1%80%D0%BE%D0%B2%D0%BE%D0%B9_%D0%BF%D1%80%D0%BE%D1%80%D1%8B%D0%B2_%D0%BD%D0%BE%D0%B2%D0%BE%D1%81%D0%B8%D0%B1%D0%B8%D1%80%D1%81%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/train_dataset_luggage/bsm_data_train.csv')

In [7]:
import pandas as pd

# Загрузка данных о потоке BSM
bsm_data_train = pd.read_csv('/content/drive/MyDrive/train_dataset_luggage/bsm_data_train.csv')

# Загрузка данных о расписании рейсов
flight_rasp_data = pd.read_csv('/content/drive/MyDrive/train_dataset_luggage/flight_rasp_data.csv')

# Загрузка данных о прогнозе прибытия
arrival_profile_data = pd.read_csv('/content/drive/MyDrive/train_dataset_luggage/arrival_profile_data.csv')
arrival_profile_value = pd.read_csv('/content/drive/MyDrive/train_dataset_luggage/arrival_profile_value.csv')

In [37]:
# Объединение по i_id
combined_data = pd.merge(bsm_data_train, flight_rasp_data, on='i_id', how='inner')

In [38]:
# Преобразование в datetime
combined_data['MessageReceivedDate'] = pd.to_datetime(combined_data['MessageReceivedDate'])
combined_data['MessageProcessedDate'] = pd.to_datetime(combined_data['MessageProcessedDate'])

# Вычисление разницы во времени
#combined_data['TimeDifference'] = combined_data['MessageProcessedDate'] - combined_data['MessageReceivedDate']

# Преобразование TimeDifference в секунды
#combined_data['TimeDifferenceSeconds'] = combined_data['TimeDifference'].dt.total_seconds()

# Расчет объема за минуту
combined_data['VolumePerMinute'] = 60 / (combined_data['MessageProcessedDate'] - combined_data['MessageReceivedDate']).dt.total_seconds()

# Вывод результата
print(combined_data[['MessageReceivedDate', 'MessageProcessedDate', 'VolumePerMinute']])

            MessageReceivedDate    MessageProcessedDate  VolumePerMinute
0       2023-07-01 06:53:34.413 2023-07-01 06:53:37.720        18.143332
1       2023-07-01 06:53:24.603 2023-07-01 06:53:28.590        15.048909
2       2023-07-01 04:31:58.227 2023-07-01 04:32:00.050        32.912781
3       2023-07-01 04:36:45.607 2023-07-01 04:36:49.783        14.367816
4       2023-07-01 04:38:30.500 2023-07-01 04:38:33.753        18.444513
...                         ...                     ...              ...
2088565 2023-08-31 23:57:31.547 2023-08-31 23:57:35.117        16.806723
2088566 2023-08-31 23:59:53.600 2023-08-31 23:59:57.260        16.393443
2088567 2023-08-31 23:58:03.420 2023-08-31 23:58:07.260        15.625000
2088568 2023-08-31 23:58:48.437 2023-08-31 23:58:49.310        68.728522
2088569 2023-08-31 23:59:05.537 2023-08-31 23:59:08.983        17.411492

[2088570 rows x 3 columns]


In [42]:
# Преобразование local_or_transfer в бинарный формат и создание нового столбца
combined_data['local_or_transfer_binary'] = combined_data['local_or_transfer'].map({'L': 1, 'T': 0})
# Удаление столбца 'local_or_transfer' с заменой
combined_data.drop(columns=['local_or_transfer'], inplace=True)

KeyError: ignored

In [44]:
combined_data['ReceivedHour'] = combined_data['MessageReceivedDate'].dt.hour
combined_data['ReceivedMinute'] = combined_data['MessageReceivedDate'].dt.minute

In [45]:
# Вывод результата
print(combined_data)

         ident_bag       i_id                       pax_id_hash  \
0         62562416  8628981.0  dcd856b334b9685c4f21afa67f3ddc70   
1         62571791  8628981.0  dcd856b334b9685c4f21afa67f3ddc70   
2         62585837  8628981.0  d1deb9ad74e154bd45625253d3192aec   
3         62585932  8628981.0  edfb80134c7e09bc74dfeead90097018   
4         62585968  8628981.0  a784df5b95726a8e444007c37de942fc   
...            ...        ...                               ...   
2088565   64616074  8687786.0  6ed0e41f175ef54a9b06f0dbab62e870   
2088566   64616154  8687786.0  2acb4b86315d6e49efc0a824b3cba80d   
2088567   64616089  8688103.0  5b4f3c75a82c28ce36d749c0700961fd   
2088568   64616120  8688009.0  57c41ae0d13db348152d36f661633ff8   
2088569   64616130  8688009.0  8a8a644086752ab32e056550927b7996   

            MessageReceivedDate    MessageProcessedDate  \
0       2023-07-01 06:53:34.413 2023-07-01 06:53:37.720   
1       2023-07-01 06:53:24.603 2023-07-01 06:53:28.590   
2       2023-07-01

In [48]:
arrival_profile_data.iloc[105888]

ts                         2023-07-01 00:00:00.000 +0300
checkin_terminal                                       B
airline_grouped_hash    189888be7b940befd5d3acecb01c5f33
pax_arrival_profile                                  0.0
Name: 105888, dtype: object

In [39]:
numeric_features = combined_data.select_dtypes(include=['float64', 'int64'])
print(numeric_features)

         ident_bag       i_id   TagNumber  config  VolumePerMinute
0         62562416  8628981.0  2425399448     189        18.143332
1         62571791  8628981.0  2425388102     189        15.048909
2         62585837  8628981.0   425411640     189        32.912781
3         62585932  8628981.0   425411641     189        14.367816
4         62585968  8628981.0   425411642     189        18.444513
...            ...        ...         ...     ...              ...
2088565   64616074  8687786.0  5115349914     138        16.806723
2088566   64616154  8687786.0  7115420782     138        16.393443
2088567   64616089  8688103.0   555433908     183        15.625000
2088568   64616120  8688009.0   555256543     158        68.728522
2088569   64616130  8688009.0   555256544     158        17.411492

[2088570 rows x 5 columns]


In [8]:
# Посмотрим на первые несколько строк данных
print(bsm_data_train.head())

# Проверим общую информацию о данных
print(bsm_data_train.info())

# Проверим наличие пропущенных значений
print(bsm_data_train.isnull().sum())

# Статистика по числовым признакам
print(bsm_data_train.describe())

# Уникальные значения в каждом столбце
for column in bsm_data_train.columns:
    unique_values = bsm_data_train[column].nunique()
    print(f'Уникальных значений в столбце {column}: {unique_values}')

# Посмотрим на распределение категориальных признаков
categorical_columns = bsm_data_train.select_dtypes(include=['object']).columns
for column in categorical_columns:
    print(f'Распределение значений в столбце {column}:')
    print(bsm_data_train[column].value_counts())

   ident_bag       i_id                       pax_id_hash  \
0   62562416  8628981.0  dcd856b334b9685c4f21afa67f3ddc70   
1   62571791  8628981.0  dcd856b334b9685c4f21afa67f3ddc70   
2   62553945  8629051.0  f2b931876cac089c6a02af1b70d08c61   
3   62553945  8629051.0  f2b931876cac089c6a02af1b70d08c61   
4   62574680  8629296.0  2e3291401628ca69061c49b3319793a1   

       MessageReceivedDate     MessageProcessedDate                DateEvent  \
0  2023-07-01 06:53:34.413  2023-07-01 06:53:37.720  2023-07-01 06:53:37.717   
1  2023-07-01 06:53:24.603  2023-07-01 06:53:28.590  2023-07-01 06:53:28.577   
2  2023-07-01 06:59:56.263  2023-07-01 06:59:59.610  2023-07-01 06:59:59.607   
3  2023-07-01 06:59:57.083  2023-07-01 06:59:59.750  2023-07-01 06:59:59.747   
4  2023-07-01 14:34:52.870  2023-07-01 14:34:55.547  2023-07-01 14:34:55.543   

    TagNumber local_or_transfer  
0  2425399448                 L  
1  2425388102                 L  
2  3555170959                 L  
3  3555170959   

Матрица корелляции двух первых файлов
Откинуть ненужные
Сделать пятиминутны интервал, плюс день
Затем добавить третий файл к объединению
и y pax